In [1]:
import sys
import copy
import time
from urllib.parse import unquote
import requests
from urllib.parse import quote
import re
from lxml import etree

## Login

In [2]:
def login(laccount, lpassword):
    """ 根据账号密码登录linkedin """
    s = requests.Session()
    r = s.get('https://www.linkedin.com/uas/login')
    tree = etree.HTML(r.content)
    loginCsrfParam = ''.join(tree.xpath('//input[@id="loginCsrfParam-login"]/@value'))
    csrfToken = ''.join(tree.xpath('//input[@id="csrfToken-login"]/@value'))
    sourceAlias = ''.join(tree.xpath('//input[@id="sourceAlias-login"]/@value'))
    isJsEnabled = ''.join(tree.xpath('//input[@name="isJsEnabled"]/@value'))
    source_app = ''.join(tree.xpath('//input[@name="source_app"]/@value'))
    tryCount = ''.join(tree.xpath('//input[@id="tryCount"]/@value'))
    clickedSuggestion = ''.join(tree.xpath('//input[@id="clickedSuggestion"]/@value'))
    signin = ''.join(tree.xpath('//input[@name="signin"]/@value'))
    session_redirect = ''.join(tree.xpath('//input[@name="session_redirect"]/@value'))
    trk = ''.join(tree.xpath('//input[@name="trk"]/@value'))
    fromEmail = ''.join(tree.xpath('//input[@name="fromEmail"]/@value'))

    payload = {
        'isJsEnabled': isJsEnabled,
        'source_app': source_app,
        'tryCount': tryCount,
        'clickedSuggestion': clickedSuggestion,
        'session_key': laccount,
        'session_password': lpassword,
        'signin': signin,
        'session_redirect': session_redirect,
        'trk': trk,
        'loginCsrfParam': loginCsrfParam,
        'fromEmail': fromEmail,
        'csrfToken': csrfToken,
        'sourceAlias': sourceAlias
    }
    s.post('https://www.linkedin.com/uas/login-submit', data=payload)
    return s

In [3]:
s = login(laccount='tsb1071463@gmail.com', lpassword='Alisa0504')

In [4]:
company_name = input('Input the company you want to crawl:')

Input the company you want to crawl:台積電


## get_Linkedin_url

In [5]:
def get_linkedin_url(url, s):
    try:
        r = s.get(url, allow_redirects=False)
        if r.status_code == 999:
            return re.findall(r'^https://tw\.linkedin\.com/in/.*',url)
    except Exception as e:
        print ('get linkedin url failed: %s' % url)
    return ''

In [6]:
def crawl(url, s):
    """ 抓取每一个搜索结果 """
    try:
        url = get_linkedin_url(href, copy.deepcopy(s))[0].replace('tw.linkedin.com', 'www.linkedin.com')
        print(url)
        if len(url) > 0 and url not in LINKS_FINISHED:
            LINKS_FINISHED.append(url)
            failure = 0
            try:
                r = s.get(url, timeout=10)
            except Exception as e:
                failure += 1
            if r.status_code != 200:
                time.sleep(1)
#                     parse(r.content, url)
#                     break
#                 else:
#                     print ('%s %s' % (r.status_code, url))
#                     failure += 2
    except Exception as e:
        pass

In [15]:
url = 'https://www.google.com/search?q=Linkedin+' + quote(company_name)+'&oq=Linkedin+'+ quote(company_name)

In [23]:
%%time
LINKS_FINISHED=[]
maxpage = 3 
failure = 0
while len(url) > 0 and failure < 10:
    try:
        res = requests.get(url,timeout=10)
        soup = bs(res.text,"lxml")
    except Exception as e:
        failure += 1
        continue
    if res.status_code == 200:
        hrefs=[i.parent['href'].split('q=')[1].split('&')[0] for i in soup.select('a > div.BNeawe.vvjwJb.AP7Wnd')]
        for href in hrefs:
            crawl(href, copy.deepcopy(s)) 
#             tree = etree.HTML(r.content)
        nextpage = soup.select('a.nBDE1b.G5eFlf')[0]['href']
        url = 'https://www.google.com' + nextpage if nextpage else ''
        failure = 0
        maxpage -= 1
        if maxpage <= 0:
            break
    elif res.status_code == 429:
        time.sleep(60)
    else:
        failure += 2
        print ('search failed: %s' % res.status_code)
if failure >= 10:
    print ('search failed: %s' % url)

KeyboardInterrupt: 

In [ ]:
from pprint import pprint 

In [ ]:
url

In [ ]:
'https://www.google.com'+soup.select('a.nBDE1b.G5eFlf')[0]['href']